In [6]:
from abc import ABC, abstractmethod


# Observable Model
class ObservableModel(ABC):
    def __init__(self):
        self.observers: list['View'] = []

    def add_observer(self, observer: 'View'):
        """Adds an observer."""
        self.observers.append(observer)

    def detach_observer(self, observer: 'View'):
        """Removes an observer."""
        if observer in self.observers:
            self.observers.remove(observer)

    @abstractmethod
    def notify_observers(self):
        """Notify all observers of changes."""
        pass


# Abstract View
class View(ABC):
    @abstractmethod
    def update(self, books: list[dict[str, str]]):
        pass


# Book Model
class BookModel(ObservableModel):
    def __init__(self):
        super().__init__()
        self._books: list[dict[str, str]] = []

    def add_book(self, title: str, author: str):
        if not title or not author:
            raise ValueError("Title and author cannot be empty.")
        if {'title': title, 'author': author} in self._books:
            raise ValueError("Book already exists.")
        self._books.append({'title': title, 'author': author})
        self.notify_observers()

    def get_books(self) -> list[dict[str, str]]:
        """Returns a copy of the books list."""
        return self._books[:]

    def notify_observers(self):
        """Notifies all observers."""
        for observer in self.observers:
            observer.update(self.get_books())


# Book View
class BookView(View):
    def update(self, books: list[dict[str, str]]):
        """Called when the model notifies observers."""
        self.display_message("Book added successfully!")
        self.display_books(books)

    def display_books(self, books: list[dict[str, str]]):
        print("Current Book List:")
        for book in books:
            print(f" - {book['title']} by {book['author']}")

    def display_message(self, message: str):
        print(message)


# Book Controller
class BookController:
    def __init__(self, model: BookModel, view: BookView):
        self.model: BookModel = model
        self.view: BookView = view
        self.model.add_observer(self.view)

    def add_book(self, title: str, author: str):
        try:
            self.model.add_book(title, author)
        except ValueError as e:
            self.view.display_message(str(e))



In [7]:
# Example Usage
if __name__ == "__main__":
    model = BookModel()
    view = BookView()
    controller = BookController(model, view)

    # Adding books
    controller.add_book("The Great Gatsby", "F. Scott Fitzgerald")
    controller.add_book("To Kill a Mockingbird", "Harper Lee")

    # Trying invalid inputs
    controller.add_book("", "Author Unknown")  # Should show an error message
    controller.add_book("The Great Gatsby", "F. Scott Fitzgerald")  # Duplicate

Book added successfully!
Current Book List:
 - The Great Gatsby by F. Scott Fitzgerald
Book added successfully!
Current Book List:
 - The Great Gatsby by F. Scott Fitzgerald
 - To Kill a Mockingbird by Harper Lee
Title and author cannot be empty.
Book already exists.


In [3]:
from typing import List, Dict


# Model
class BookModel:
    def __init__(self):
        self._books: List[Dict[str, str]] = []

    def add_book(self, title: str, author: str):
        if not title or not author:
            raise ValueError("Title and author cannot be empty")
        if {'title': title, 'author': author} in self._books:
            raise ValueError("Duplicate book entry")
        self._books.append({'title': title, 'author': author})

    def get_books(self) -> List[Dict[str, str]]:
        return self._books[:]


# View Interface
class BookViewInterface(ABC):
    @abstractmethod
    def display_message(self, message: str):
        pass

    @abstractmethod
    def display_books(self, books: List[Dict[str, str]]):
        pass
    

# View
class BookView(BookViewInterface):
    def display_message(self, message: str):
        print(message)

    def display_books(self, books: List[Dict[str, str]]):
        print("Current Book List:")
        for book in books:
            print(f"Book: {book['title']} by {book['author']}")


# Presenter
class BookPresenter:
    def __init__(self, model: BookModel, view: BookViewInterface):
        self.model = model
        self.view = view

    def add_book(self, title: str, author: str):
        try:
            self.model.add_book(title, author)
            self.view.display_message("Book added successfully!")
        except ValueError as e:
            self.view.display_message(str(e))
        self.view.display_books(self.model.get_books())


In [4]:
# Instantiate the components
model = BookModel()
view = BookView()
presenter = BookPresenter(model, view)
    # Adding books
presenter.add_book("The Great Gatsby", "F. Scott Fitzgerald")
presenter.add_book("To Kill a Mockingbird", "Harper Lee")

# Trying invalid inputs
presenter.add_book("", "Author Unknown")  # Should show an error message
presenter.add_book("The Great Gatsby", "F. Scott Fitzgerald")  # Duplicate


Book added successfully!
Current Book List:
Book: The Great Gatsby by F. Scott Fitzgerald
Book added successfully!
Current Book List:
Book: The Great Gatsby by F. Scott Fitzgerald
Book: To Kill a Mockingbird by Harper Lee
Title and author cannot be empty
Current Book List:
Book: The Great Gatsby by F. Scott Fitzgerald
Book: To Kill a Mockingbird by Harper Lee
Duplicate book entry
Current Book List:
Book: The Great Gatsby by F. Scott Fitzgerald
Book: To Kill a Mockingbird by Harper Lee
